In [ ]:
import os
import json
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai #This is how I finally got both to work: system_instruction and stream
import anthropic
from IPython.display import Markdown
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import gradio as gr

In [ ]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

PATH_TO_CHROME_DRIVER = 'B:\\Users\\ekfon\\chromeDriver\\chromedriver.exe'

In [ ]:
openai = OpenAI()
claude = anthropic.Anthropic()
genai.configure(api_key=google_api_key) #for gemini

In [ ]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

force_light_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'light') {
        url.searchParams.set('__theme', 'light');
        window.location.href = url.href;
    }
}
"""

In [ ]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        page_source = driver.page_source
        driver.quit()
        
        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage title: \"{self.title}\"\nWebpage contents:\n{self.text}\n\n"

In [ ]:
#system prompt for the link anthology
anthology_sPrompt = "I'll provide you with a list of links from a webpage. \
You are able to decide which links are most relevant to include in a brochure about the company, \
such as the About page, any Company page, or a jobs/careers page.\n"

anthology_sPrompt += "You will respond in JSON format, providing full https URLs, just like in this example:\n"

anthology_sPrompt += """
{
    "links": [
        {"type": "about page", "url": "https://www.example-url.com/about"}
        {"type": "careers page", "url": "https://further.example-url.co.uk/Careers/"}
    ]
}
"""

In [ ]:
def get_anthology_user_prompt(website):
    user_prompt = f"Below is the list of links from the webpage {website.url}. "
    user_prompt += "Please decide which of the links are relevant for a brochure about the company. \
Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links."
    user_prompt += "Here is the list of links (some might be relative links):\n\n"
    user_prompt += "\n".join(website.links)

    return user_prompt

In [ ]:
def get_links_anthology_gpt(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": anthology_sPrompt},
            {"role": "user", "content": get_anthology_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result) #because result is a string, and what we want is an actual dictionary

In [ ]:
def get_links_anthology_claude(url):
    website = Website(url)
    response = claude.messages.create(
        model="claude-3-haiku-20240307",
        system=anthology_sPrompt,
        messages=[{"role": "user", "content": get_anthology_user_prompt(website)}],
        max_tokens=500
    )
    result = response.content[0].text
    return json.loads(result) #because result is a string, and what we want is an actual dictionary

In [ ]:
def get_links_anthology_gemini(url):
    website = Website(url)
    model = genai.GenerativeModel(
        "models/gemini-2.0-flash",
        system_instruction=anthology_sPrompt + "Do not comment on your answer. Do not put ``` around your answer.",
    )
    response = model.generate_content(get_anthology_user_prompt(website))
    result = response.text
    return json.loads(result.replace("```json\n","").replace("\n```","")) #Gemini sometimes does this

In [ ]:
def get_all_details(url, model):
    result = "Landing page:\n\n"
    result += Website(url).get_contents()
    if model == 'GPT':
        links = get_links_anthology_gpt(url)
    elif model == 'Claude':
        links = get_links_anthology_claude(url)
    else:
        links = get_links_anthology_gemini(url)

    for link in links["links"]: #remember that links is a json dictionary
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()

    return result

In [ ]:
brochure_sPrompt = "You analyze the content of several relevant pages from a company's website. \
You use that knowledge to create a short brochure about the company. Your brochure is for prospective customers, investors, and recruits. \
Include details of company culture, customers, and job openings if you have the information. Respond in Markdown."

In [ ]:
def get_brochure_user_prompt(company_name, url, model):
    prompt = f"You are looking at the website of the company called {company_name}.\n"
    prompt += "Here are the contents of its landing page and other relevant pages. Based on this content, \
create a short brochure of the company in Markdown:\n\n"
    prompt += get_all_details(url, model)
    prompt = prompt[:5_000] #this limits the prompt input, just in case
    
    return prompt

In [ ]:
def stream_gpt_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
        {"role": "system", "content": brochure_sPrompt},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url, 'GPT')}
        ],
        stream=True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        yield response

In [ ]:
def stream_claude_brochure(company_name, url):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=brochure_sPrompt,
        messages=[
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, 'Claude')},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
def stream_gemini_brochure(company_name, url):
    model = genai.GenerativeModel(
        "models/gemini-2.0-flash",
        system_instruction=brochure_sPrompt + "Do not comment on your answer. Do not put ``` around your answer.",
    )    
    response = model.generate_content(
        get_brochure_user_prompt(company_name, url, 'Gemini'),
        stream=True
    )    
    result = ""
    for chunk in response:
        result += chunk.text or ""
        yield result

In [ ]:
def stream_model_brochure(model, company_name, url):
    if model=="GPT":
        result = stream_gpt_brochure(company_name, url)
    elif model=="Claude":
        result = stream_claude_brochure(company_name, url)
    elif model=="Gemini":
        result = stream_gemini_brochure(company_name, url)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model_brochure,
    inputs=[gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT"), gr.Textbox(label="Company name:"), gr.Textbox(label="Website URL:")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch(inbrowser=True)